In [4]:
import pandas as pd
import numpy as np
import pickle
import torch
from torch.nn import Sequential, ReLU, Module, Dropout, Sigmoid, Linear, BatchNorm2d
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import torchvision.utils as vutils
import pickle
import os
import matplotlib.pyplot as plt

In [16]:
data = ImageFolder(root='../Handwritten-Equation-Solver/data/character_data')
classes = os.listdir('../Handwritten-Equation-Solver/data/character_data')

for i in range(len(classes)):
    if(classes[i][0] == "'" and classes[i][2] == "'"):
        classes[i] = classes[i][1]
print(len(classes))

107


In [ ]:
'''
Two Convolutional Layers
Two MaxPooling Layers
Dropout Layer
Flatten Layer
Two Dense Layers using Relu
Output Layer using Softmax
Adam Optimizer
'''